In [ ]:
# For youtube transcription Use the version of above 1.1
from youtube_transcript_api import YouTubeTranscriptApi,TranscriptsDisabled
def yttranscript(video_id,lang='en'):
    try:
    # If you don’t care which language, this returns the “best” one
        transcript_list = YouTubeTranscriptApi().fetch(video_id, languages=["en"])

    # Flatten it to plain text
        transcript = " ".join(chunk.text for chunk in transcript_list)
        return transcript

    except TranscriptsDisabled:
        print("No captions available for this video.")


In [18]:
text=yttranscript(video_id="JurOkE1p7Ug")

In [19]:
print(text)

A new study has come out by Microsoft which talks about the 40 jobs that are most likely to be taken by AI in the near future and also 40 jobs that are relatively safe. Now you might ask me why don't if a big company has put out a paper like this why would you listen to them? They have so much incentive to lie. But the truth is actually in this case Microsoft has all the incentives not to put out a paper like this simply because of the amount of fear and panic it'll create. And if you actually go through the list of these jobs, you'll see that it's very risky for Microsoft as a company to put this out because a lot of jobs that they say are going to be replaced are jobs that in a way their tool suite enables. So I wanted to actually go through the 40 jobs that are likely to be replaced and also the 40 jobs that are safe. Without much further ado, let's go to the 40 jobs that are first going to be replaced. Before we begin, I want to tell you the methodology used. They took about 200,00

In [20]:
from urllib.parse import urlparse, parse_qs

def extract_video_id(url: str) -> str:
    """
    Extracts the YouTube video ID from a given URL.
    Supports both 'youtube.com' and 'youtu.be' formats.
    """
    parsed_url = urlparse(url)

    # Case 1: https://www.youtube.com/watch?v=VIDEO_ID
    if parsed_url.hostname in ["www.youtube.com", "youtube.com"]:
        query = parse_qs(parsed_url.query)
        return query.get("v", [None])[0]

    # Case 2: https://youtu.be/VIDEO_ID
    if parsed_url.hostname == "youtu.be":
        return parsed_url.path.lstrip("/")

    return None


In [21]:
def yttranscript_from_url(url, lang="en"):
    video_id = extract_video_id(url)
    if not video_id:
        return "Invalid YouTube URL"
    return yttranscript(video_id, lang)


In [23]:
text=yttranscript_from_url("https://www.youtube.com/watch?v=JurOkE1p7Ug")

In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([text])

In [25]:
chunks

[Document(metadata={}, page_content="A new study has come out by Microsoft which talks about the 40 jobs that are most likely to be taken by AI in the near future and also 40 jobs that are relatively safe. Now you might ask me why don't if a big company has put out a paper like this why would you listen to them? They have so much incentive to lie. But the truth is actually in this case Microsoft has all the incentives not to put out a paper like this simply because of the amount of fear and panic it'll create. And if you actually go through the list of these jobs, you'll see that it's very risky for Microsoft as a company to put this out because a lot of jobs that they say are going to be replaced are jobs that in a way their tool suite enables. So I wanted to actually go through the 40 jobs that are likely to be replaced and also the 40 jobs that are safe. Without much further ado, let's go to the 40 jobs that are first going to be replaced. Before we begin, I want to tell you the met

In [27]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\Admin\Desktop\MyDesktop\Langchain\myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Admin\Desktop\MyDesktop\Langchain\myvenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python a

In [29]:
from langchain_community.vectorstores import FAISS

vectorstore=FAISS.from_documents(chunks, embedding)

In [53]:
vectorstore.index_to_docstore_id[0]

'bd13b48a-b8a8-4c72-99ef-91ada7f3cb09'

In [54]:
vectorstore.get_by_ids([vectorstore.index_to_docstore_id[0]])

[Document(id='bd13b48a-b8a8-4c72-99ef-91ada7f3cb09', metadata={}, page_content="A new study has come out by Microsoft which talks about the 40 jobs that are most likely to be taken by AI in the near future and also 40 jobs that are relatively safe. Now you might ask me why don't if a big company has put out a paper like this why would you listen to them? They have so much incentive to lie. But the truth is actually in this case Microsoft has all the incentives not to put out a paper like this simply because of the amount of fear and panic it'll create. And if you actually go through the list of these jobs, you'll see that it's very risky for Microsoft as a company to put this out because a lot of jobs that they say are going to be replaced are jobs that in a way their tool suite enables. So I wanted to actually go through the 40 jobs that are likely to be replaced and also the 40 jobs that are safe. Without much further ado, let's go to the 40 jobs that are first going to be replaced. 

In [32]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [33]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019AC1D70C20>, search_kwargs={'k': 4})

In [34]:
retriever.invoke("what is 1 st job to be replaced.")

[Document(id='6b04a566-a995-496d-9459-f5aa0b087fae', metadata={}, page_content="and also the 40 jobs that are safe. Without much further ado, let's go to the 40 jobs that are first going to be replaced. Before we begin, I want to tell you the methodology used. They took about 200,000 anonymous conversations from copilot of people asking copilot to help them with their work. And then they looked at the work that copilot did and tried to figure out did it actually do the work end to end or not. Then depending on the user satisfaction of that work done by copilot they decided whether AI was applicable there or not. And at the end they created this AI applicability score for each of these jobs which is how likely is it that AI can do it end to end in the next few months or years. Anyway that was the methodology. Let's get straight to the 40 jobs that AI can replace. Number one is interpreters and translators. Now, this makes obvious sense, right? The transformer as a technology was initial

In [35]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()


# 2. Initialize Gemini 2.5 Pro
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.7,
)



In [36]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)

In [43]:
question          = "what is 1st job to be replaced."
retrieved_docs    = retriever.invoke(question)

In [44]:
retrieved_docs

[Document(id='e2fda6c5-4299-4fc1-b903-489beb49cd0a', metadata={}, page_content="waste or removes hazardous material that none of us want to do is the kind of jobs that will be protected from AI. All the cushion cushy jobs sitting in front of a computer like mine are going to be automated while all the jobs that make society run properly continue to exist. I remember saying this on a podcast. Tomorrow if I had to be a petrol pump assistant, somebody sitting in a petrol pump putting in petrol or a garbage collector and that was the only way to survive, I'd do it. I'm very adaptable like that. I have very little attachment to my current status. I will do whatever is necessary to survive. And when you've been an entrepreneur for 12 years of your life, you don't have a choice. It's almost like the market will make you do something to survive and you got to do it to survive. So I have less addiction to specific titles or job roles. I care more about uh how how can I continue to exist and if 

In [45]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"waste or removes hazardous material that none of us want to do is the kind of jobs that will be protected from AI. All the cushion cushy jobs sitting in front of a computer like mine are going to be automated while all the jobs that make society run properly continue to exist. I remember saying this on a podcast. Tomorrow if I had to be a petrol pump assistant, somebody sitting in a petrol pump putting in petrol or a garbage collector and that was the only way to survive, I'd do it. I'm very adaptable like that. I have very little attachment to my current status. I will do whatever is necessary to survive. And when you've been an entrepreneur for 12 years of your life, you don't have a choice. It's almost like the market will make you do something to survive and you got to do it to survive. So I have less addiction to specific titles or job roles. I care more about uh how how can I continue to exist and if I have to be a hazardous materials removal worker tomorrow so be it. Next is\n\

In [46]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [47]:
final_prompt

StringPromptValue(text="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      waste or removes hazardous material that none of us want to do is the kind of jobs that will be protected from AI. All the cushion cushy jobs sitting in front of a computer like mine are going to be automated while all the jobs that make society run properly continue to exist. I remember saying this on a podcast. Tomorrow if I had to be a petrol pump assistant, somebody sitting in a petrol pump putting in petrol or a garbage collector and that was the only way to survive, I'd do it. I'm very adaptable like that. I have very little attachment to my current status. I will do whatever is necessary to survive. And when you've been an entrepreneur for 12 years of your life, you don't have a choice. It's almost like the market will make you do something to survive and you got to do it to survive. So I ha

In [48]:
answer = llm.invoke(final_prompt)
print(answer.content)

According to the transcript, the first job that AI can replace is "interpreters and translators."


In [49]:
from app import add

In [50]:
add(1,2)

3